In [185]:
import time
import itertools
import string
import random
import Levenshtein
import networkx as nx
from scipy.sparse import coo_matrix
from sys import argv
from math import factorial
random.seed(1)
start_time = time.clock()

minimum_aa = 4
maximum_aa = 20
min_seqs = 0
max_seqs = 50000

In [29]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [146]:
print 'Number of iterations will be %d'%(factorial(max_seqs)/2/(factorial(max_seqs-2)))

Number of iterations will be 1249975000


In [147]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
import os, sys

os.environ['PYSPARK_PYTHON'] = sys.executable

conf = SparkConf()

In [5]:
conf.set('spark.executor.instances',  100)
conf.set('spark.executor.cores', 4)
conf.set('spark.executor.memory', '9g')

In [6]:
sc = SparkContext(master='yarn-client', conf=conf)

In [186]:
string_list = []

for i in range(min_seqs, max_seqs):
    random_integer = random.randrange(minimum_aa, maximum_aa)
    s = ''.join(random.choice('ACDEFGHIKLMNPQRSTVWY') for _ in range(random_integer))
    string_list.append(s)

d = {}
d_i = {}
for i, s in enumerate(string_list):
    d[i] = s
    d_i[s] = i

In [202]:
Npart = 400
n_per = max_seqs/Npart/2

def balance_partitions(x): 
    part = 0
    if x < max_seqs/2:
        part = x/n_per
    else:
        part = (max_seqs-x)/n_per
    return part

In [203]:
idx_rdd = sc.parallelize(xrange(max_seqs), Npart).map(lambda x: (x,None)).partitionBy(Npart, balance_partitions).keys()
string_rdd = sc.parallelize(string_list, 2000)

In [190]:
strings_b = sc.broadcast(string_list)
d_b = sc.broadcast(d)
d_i_b = sc.broadcast(d_i)

In [191]:
def generate_matrix_elements_idx(iterator):
    string_map = d_b.value
    
    for my_idx in iterator: 
        for idx in range(my_idx):
            ld = Levenshtein.distance(string_map[my_idx], string_map[idx])
            if ld < 2 and ld > 0: 
                yield (my_idx, idx, float(ld))

In [192]:
idx_rdd.cache().count()

50000

In [206]:
idx_rdd.mapPartitions(get_len).max()

6200000

In [207]:
%%time 
mat = idx_rdd.mapPartitions(generate_matrix_elements_idx)
print mat.count()

2718
CPU times: user 33 ms, sys: 15 ms, total: 48 ms
Wall time: 8.26 s


In [ ]:
def get_matrix_elements(string_list):
    levenshtein_list = []
    rows = []
    cols = []

    for a, b in itertools.combinations(string_list, 2):
        levenshtein_distance = Levenshtein.distance(a, b)
        if levenshtein_distance < 2:
            levenshtein_list.append(1)
            rows.append(d_i[a])
            cols.append(d_i[b])
    return levenshtein_list, rows, cols

In [ ]:
nx.write_gml(g, "g_500K.gml")


text_file = open("output_500K.txt", "w")
text_file.write(' '.join(('Number of generated sequences:', str(len(string_list)), 'sequences'))) 
text_file.write(' '.join(('\nTime calculating the Levenshtein distance:', str(dist_time), 'seconds')))
text_file.write(' '.join(('\nTime calculating the graph:', str(graph_time), 'seconds')))
text_file.write(' '.join(('\nTotal pipeline time:', str(total_time), 'seconds')))
text_file.write('\nSuccessful finish')
text_file.close()